# Introduction to the Evaluation Class

## Overview
In the previous lesson we loaded some test data from the user's local drive (although we first downloaded it from the repository). In this example we will continue to explore the Evaluation schema through the Evaluation class interface. 

Note: this lesson builds off of the dataset that we created in the last lesson `Loading Local Data`.  If you have not run through the Loading Local Data lesson, then go back and first work though that notebook to generate the required dataset for this lesson.

### Create a new Evaluation
First we will import the the TEEHR Evaluation class and create a new instance that points to a directory where the data loaded in lesson `02_loading_data` is stored.

In [ ]:
import teehr
from teehr.evaluation.utils import print_tree
from pathlib import Path

# Define the directory where the Evaluation will be created
test_eval_dir = Path(Path().home(), "temp", "02_loading_data")

# Create an Evaluation object and create the directory
ev = teehr.Evaluation(dir_path=test_eval_dir, create_dir=True)

Now that we have created a new evaluation that points to the dataset we created in `02_loading _data`, lets take a look at the data, specifically the `dataset` directory.  You can see that the three different data groups are stored in slightly different ways.  
- The domain tables (units, variables, configurations, attributes) are stored as *.csv files.  While in this case the files happen to have the same name as the table, there is no requirement that they do.
- The location tables (locations, location_attributes, location_crosswalks) are stored as parquet files without hive partitioning.  The file names are managed by Spark.
- The timeseries tables (primary_timeseries, secondary_timeseries, joined_timeseries) are stored as parquet files with hive partitioning. The file names are managed by Spark.

Note, if you don't have tree installed and don't want to install it, you can uncomment the comment lines to use a Python function roughly does the same thing.

In [ ]:
print_tree(ev.dataset_dir, exclude_patterns=[".*", "_*"])

### Table Classes
The TEEHR Evaluation class contains different sub-classes that are used to organize class methods into logical groups.  One of these types of sub-classes is the "table" sub-classes which contain methods for interacting with the data tables. Each of the tables in the Evaluation dataset has a respective sub-class with the table name.
```
ev.units
ev.attributes
ev.variables
ev.configurations
ev.locations
ev.location_attributes
ev.location_crosswalks
ev.primary_timeseries
ev.secondary_timeseries
ev.joined_timeseries
```
Each of the table sub-classes then has methods to add and/or load new data as well as methods to query the table to get data out.  These are documented in the API documentation. For now, because all the tables are relatively small, we will just use the `to_pandas()` method and then the `head()` method on the Pandas DataFrame to see an example of the data that is returned.  In an actual evaluation setting, with lots of data in the TEEHR dataset, you would likely want to include a `filter()` method to reduce the amount of data you are querying and putting into memory.

In [ ]:
ev.units.to_pandas().head()

In [ ]:
ev.attributes.to_pandas().head()

In [ ]:
ev.variables.to_pandas().head()

In [ ]:
ev.configurations.to_pandas().head()

In [ ]:
ev.locations.to_pandas().head()

In [ ]:
ev.location_attributes.to_pandas().head()

In [ ]:
ev.primary_timeseries.to_pandas().head()

In [ ]:
ev.location_crosswalks.to_pandas().head()

In [ ]:
ev.secondary_timeseries.to_pandas().head()

### Querying
Above, we just used the `to_pandas()` method on each table in the dataset to see an example of the data that is in each table. The underlying query engine for TEEHR is PySpark.  As a result, each of the table sub-classes can return data as either a Spark DataFrame (using the `to_sdf()` method) or as a Pandas DataFrame (using the `to_pandas()` method).  The location data tables have an additional method that returns a GeoPandas DataFrame (using the `to_geopandas()` method) where the geometry bytes column has been converted to a proper WKT geometry column.

Note: PySpark itself is "lazy loaded" meaning that it does not actually run the query until the data is needed for display, plotting, etc.  Therefore, if you just use the `to_sdf()` method, you do not get the data but rather a lazy Spark DataFrame that can be used with subsequent Spark operations that will all be evaluated when the results are requested.  Here we show how to get the Spark DataFrame and show the data but there are many other ways that the lazy Spark DataFrame can be used in subsequent operations that are beyond the scope of this document.

In [ ]:
# Query the locations and return as a lazy Spark DataFrame.
ev.locations.to_sdf()

In [ ]:
# Query the locations and return as a Spark DataFrame but tell Spark to show the data.
ev.locations.to_sdf().show()

In [ ]:
# Query the locations and return as a Pandas DataFrame.
# Note that the geometry column is shown as a byte string.
ev.locations.to_pandas()

In [ ]:
# Query the locations and return as a GeoPandas DataFrame.
# Note that the geometry column is now a proper WKT geometry column.
ev.locations.to_geopandas()

One very quick example of how the Spark DataFrame's lazy loading can be beneficial, would be to get the number of rows in a query result.  If you did `len(ev.primary_timeseries.to_pandas())`, first the entire data frame would have to be loaded in memory as a Pandas DataFrame and then the length calculated.  On the otherhand, if you were to `ev.primary_timeseries.to_sdf().count()` the Spark engine would calculate the number of rows without loading the entire dataset into memory first.  For larger datsets this could be very important.

In [ ]:
display(
    len(ev.primary_timeseries.to_pandas())
)
display(
    ev.primary_timeseries.to_sdf().count()
)

### Filtering and Ordering
As noted above, because the tables are a lazy loaded Spark DataFrames, we can filter and order the data before returning it as a Pandas or GeoPandas DataFrame. The filter methods take either a raw SQL string, a filter dictionary or a FilterObject, Operator and field enumeration. Using an FilterObject, Operator and field enumeration is probably not a common pattern for most users, but it is used internally to validate filter arguments and is available to users if they would like to use it.

In [ ]:
# Filter using a raw SQL string
ev.locations.filter("id = 'usgs-A'").to_geopandas()

In [ ]:
# Filter using a dictionary
ev.locations.filter({
    "column": "id",
    "operator": "=",
    "value": "usgs-A"
}).to_geopandas()

In [ ]:
# Import the LocationFilter and Operators classes
from teehr import LocationFilter, Operators

# Get the field enumeration
fields = ev.locations.field_enum()

# Filter using the LocationFilter class
lf = LocationFilter(
    column=fields.id,
    operator=Operators.eq,
    value="usgs-A"
)
ev.locations.filter(lf).to_geopandas()

In [ ]:
ev.spark.stop()

This same approach can be used to query the other tables in the evaluation dataset.  There are also other methods that we did not explore and users are encouraged to checkout the TEEHR API documentation as well as the PySpark documentation for a more in-depth understanding of what happens in the background.